# Final Recommender

THIS NOTEBOOK IS STILL IN PROGRESS - THIS IS THE HARDEST CODE I'VE EVER HAD TO WRITE. WILL BE UP TONIGHT

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import os
from surprise import Dataset
from surprise import Reader
from surprise import accuracy

#matrix factorization
from surprise import SVDpp

#matrix factorization
from surprise import KNNBasic
from surprise import KNNBaseline

In [2]:
#notify me when a long running cell is complete
%load_ext jupyternotify
import time

<IPython.core.display.Javascript object>

# The Data

In [49]:
data = pd.read_csv('data/1m_useratt_minreq.csv')
minorityrec = pd.read_csv('data/minreq.csv')

#for split
data['r_date'] = data['r_date'].astype('datetime64[ns]')

data.head()

,mid,cust_id,rating,r_date,m_decade,m_avg_rating,user_engagement,cust_act_activity_rank,adopters,m_minreq
0,3113,510180,3.0,1999-12-12,4,3.187192,9,4,1,0.0
1,3113,1589382,4.0,2000-03-11,4,3.187192,2,2,1,0.0
2,3113,1878798,2.0,2000-08-08,4,3.187192,30,5,1,0.0
3,3113,1259176,4.0,2000-11-10,4,3.187192,7,4,1,0.0
4,3113,873369,3.0,2000-11-20,4,3.187192,9,4,1,0.0


In [50]:
minorityrec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17770 entries, 0 to 17769
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   mid       17770 non-null  int64  
 1   m_minreq  17770 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 277.8 KB


# The Models

In [4]:
reader = Reader(rating_scale=(1,5))
all_data = Dataset.load_from_df(data[['cust_id','mid','rating']], reader)

#correct surprise dataset format
all_sr = all_data.build_full_trainset()

In [5]:
#instantiate best Matrix Factorization Model and fit to all data
svdpp = SVDpp(n_factors= 150, n_epochs= 20)
svdpp.fit(all_sr)

In [83]:
#instantiate best kNN Model and fit to all data
sim_dict = {'name': 'cosine', 'user_based': False}
knn = KNNBaseline(min_k=5, sim_options=sim_dict)
knn.fit(all_sr)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


# Intentional Diversity

A curated list from the celebrities below:

**Will Smith:**
- Hitch (2005): 17324
- Shark Tale (2004): 5345
- Bad Boys (1995): 2186

**Lucy Liu:**
- Mulan 2 (2004): 13836
- Charlie's Angels (2000): 6552

**Jennifer Lopez:**
- Maid in Manhattan (2002): 11149
- Out of Sight (1998): 13486

**Holdout Will Smith Movie:**
- Men In Black (2002): 12918

In [94]:
12918 in data['mid'].unique()

True

In [80]:
diverse_mids = [17324, 5345, 2186, 13836, 6552, 11149, 13486, 12918]

In [100]:
def nn_iid_dict(mid_list):
    
    nn_dict = {}
    
    for mid in mid_list:
        #inner id
        mid_iid = knn.trainset.to_inner_iid(mid)
        
        #top 20 neighbors
        neighbors = knn.get_neighbors(mid_iid, k=20)
        
        #add to dictionary
        nn_dict[mid] = neighbors
        
    return nn_dict

In [102]:
diverse_dict = nn_iid_dict(diverse_mids)
print(diverse_dict)

{17324: [4, 11, 14, 16, 23, 24, 26, 37, 45, 47, 56, 63, 66, 71, 78, 79, 80, 85, 89, 93], 5345: [1, 7, 11, 12, 17, 20, 24, 33, 37, 46, 51, 58, 60, 63, 65, 71, 72, 81, 86, 95], 2186: [0, 5, 22, 24, 36, 45, 52, 57, 58, 74, 75, 78, 82, 88, 90, 98, 100, 103, 104, 109], 13836: [1, 7, 8, 19, 48, 74, 77, 78, 85, 92, 103, 106, 117, 122, 124, 132, 154, 164, 167, 173], 6552: [0, 1, 11, 20, 36, 37, 43, 48, 49, 52, 54, 56, 59, 60, 64, 66, 67, 68, 72, 73], 11149: [2, 7, 15, 17, 43, 47, 50, 62, 68, 71, 87, 88, 93, 96, 97, 114, 116, 126, 128, 129], 13486: [1, 4, 7, 23, 39, 41, 42, 45, 50, 59, 63, 66, 70, 72, 76, 79, 81, 84, 88, 90]}


In [129]:
cust_id = user1

def diversity_check(initial_recs_df):
    """
    
    """
    #list of mids in initial rec df
    mid_list = list(initial_recs_df['mid'].unique())

    #instantiate list of 10nn for each rec
    hundred_initial_rec_nn = []

    #nearest neighbors for initial recs_df
    def nn_rec_mids(mid_list):

        def nn(mid):
            #inner id
            mid_iid = knn.trainset.to_inner_iid(mid)

            #top 10 neighbors
            return knn.get_neighbors(mid_iid, k=10)

    #apply nn_rec_mids and + to list
        ten_initial_nn = nn_rec_mids(mid_list)
        hundred_initial_rec_nn.extend(ten_initial_nn)

        return hundred_initial_rec_nn
    
    def diversity(hundred_initial_rec_nn):

        #diversity check Hitch(2005) 17324
        if knn.trainset.to_inner_iid(17324) in hundred_initial_rec_nn:
            newrow = cust_id, 17324, 1

        #diversity check Mulan 2 13836
        elif knn.trainset.to_inner_iid(13836) in hundred_initial_rec_nn:
            newrow = cust_id, 113836, 1

        #diversity check Maid In Manhattan 11149
        elif knn.trainset.to_inner_iid(11149) in hundred_initial_rec_nn:
            newrow = cust_id, 11149, 1

        #diversity check Shark Tale 5345
        elif knn.trainset.to_inner_iid(5345) in hundred_initial_rec_nn:
            newrow = cust_id, 5345, 1

        #diversity check Charlie's Angels 6552
        elif knn.trainset.to_inner_iid(6552) in hundred_initial_rec_nn:
            newrow = cust_id, 6552, 1

        #diversity check Out of Sight 13486
        elif knn.trainset.to_inner_iid(13486) in hundred_initial_rec_nn:
            newrow = cust_id, 13486, 1

        #diversity check Bad Boys 2186
        elif knn.trainset.to_inner_iid(2186) in hundred_initial_rec_nn:
            newrow = cust_id, 2186, 1

        #else return MIB 12918
        else:
            new_row = cust_id, 12918, 1

        #insert new row at index 3
        initial_recs_df.loc[3] = new_row
        initial_recs_df = initial_recs_df.sort_index().reset_index(drop=True)
        return initial_recs_df[:10]

In [130]:
dchecktest = diversity_check(pvr1)
dchecktest

NameError: name 'new_row' is not defined

# User #2407458: DONE BY HAND FOR RIGHT NOW

Throughout this process, I have been tracking the recommendations given to this user

In [125]:
user1 = data[data['cust_id'] == 2407458]
user1

,mid,cust_id,rating,r_date,m_decade,m_avg_rating,user_engagement,cust_act_activity_rank,adopters,m_minreq
54936,15342,2407458,3.0,2005-11-17,4,3.476331,4,3,5,0.0
76944,16128,2407458,4.0,2005-11-05,4,3.964478,4,3,5,0.0
130508,14606,2407458,3.0,2005-11-05,5,3.124744,4,3,5,0.0
698884,4157,2407458,3.0,2005-11-17,4,3.357143,4,3,5,1.0


In [194]:
top15_svdpp_allusers = pd.read_csv('data/top15_svdpp_allusers.csv')
svdpp_example = top15_svdpp_allusers[(top15_svdpp_allusers['cust_id'] == 2407458)]
svdpp_example

,cust_id,mid,m_minreq
267998,2407458,12834,0.0
502332,2407458,7230,0.0
864390,2407458,10418,0.0
997165,2407458,16587,0.0
1151834,2407458,14621,0.0
1558123,2407458,12293,0.0
1713263,2407458,2102,0.0
2183369,2407458,5103,0.0
2553679,2407458,14691,0.0
2824970,2407458,10080,0.0


In [195]:
svdpp_example = svdpp_example[:10].reset_index()
svdpp_example

,index,cust_id,mid,m_minreq
0,267998,2407458,12834,0.0
1,502332,2407458,7230,0.0
2,864390,2407458,10418,0.0
3,997165,2407458,16587,0.0
4,1151834,2407458,14621,0.0
5,1558123,2407458,12293,0.0
6,1713263,2407458,2102,0.0
7,2183369,2407458,5103,0.0
8,2553679,2407458,14691,0.0
9,2824970,2407458,10080,0.0


In [139]:
svdpp_list = list(svdpp_example['mid'])
svdpp_list

[12834, 7230, 10418, 16587, 14621, 12293, 2102, 5103, 14691, 10080]

In [174]:
list1 = []

for mid in svdpp_list:
    mid_iid = knn.trainset.to_inner_iid(mid)
    ten = knn.get_neighbors(mid_iid, k=10)
    list1.extend(ten)

In [175]:
list1

[1,
 6,
 15,
 16,
 40,
 53,
 56,
 63,
 64,
 79,
 3,
 7,
 9,
 13,
 14,
 22,
 26,
 29,
 38,
 41,
 0,
 8,
 10,
 17,
 22,
 24,
 25,
 30,
 36,
 44,
 4,
 7,
 8,
 11,
 22,
 23,
 25,
 36,
 39,
 40,
 5,
 15,
 24,
 30,
 31,
 37,
 50,
 51,
 53,
 54,
 0,
 4,
 5,
 9,
 21,
 22,
 46,
 51,
 57,
 58,
 0,
 4,
 9,
 13,
 23,
 30,
 37,
 44,
 46,
 56,
 1,
 7,
 11,
 20,
 24,
 29,
 30,
 40,
 45,
 48,
 1,
 6,
 7,
 8,
 14,
 15,
 16,
 22,
 30,
 47,
 3,
 7,
 9,
 20,
 29,
 36,
 39,
 40,
 62,
 67]

In [177]:
len(list1)

100

In [179]:
hitch = knn.trainset.to_inner_iid(17324)
hitch

488

In [180]:
knn.trainset.to_inner_iid(17324) in list1

False

In [181]:
mulan = knn.trainset.to_inner_iid(13836)
knn.trainset.to_inner_iid(13836) in list1

False

In [182]:
knn.trainset.to_inner_iid(11149) in list1

False

In [183]:
knn.trainset.to_inner_iid(5345) in list1

False

In [184]:
knn.trainset.to_inner_iid(6552) in list1

False

In [185]:
knn.trainset.to_inner_iid(13486) in list1

False

In [186]:
knn.trainset.to_inner_iid(2186) in list1

False

In [198]:
new_row = [user1, 12918, 1]
#svdpp_example = svdpp_example.append(new_row, ignore_index=True)
svdpp_example.loc[3, :] = [0, 2407458, 12918, 1]
svdpp_example

,index,cust_id,mid,m_minreq
0,267998,2407458,12834,0.0
1,502332,2407458,7230,0.0
2,864390,2407458,10418,0.0
3,0,2407458,12918,1.0
4,1151834,2407458,14621,0.0
5,1558123,2407458,12293,0.0
6,1713263,2407458,2102,0.0
7,2183369,2407458,5103,0.0
8,2553679,2407458,14691,0.0
9,2824970,2407458,10080,0.0


In [ ]:
def diversity(initial_recs_df, hundred_initial_rec_nn):

        #diversity check Hitch(2005) 17324
        if knn.trainset.to_inner_iid(17324) in hundred_initial_rec_nn:
            newrow = cust_id, 17324, 1

        #diversity check Mulan 2 13836
        elif knn.trainset.to_inner_iid(13836) in hundred_initial_rec_nn:
            newrow = cust_id, 113836, 1

        #diversity check Maid In Manhattan 11149
        elif knn.trainset.to_inner_iid(11149) in hundred_initial_rec_nn:
            newrow = cust_id, 11149, 1

        #diversity check Shark Tale 5345
        elif knn.trainset.to_inner_iid(5345) in hundred_initial_rec_nn:
            newrow = cust_id, 5345, 1

        #diversity check Charlie's Angels 6552
        elif knn.trainset.to_inner_iid(6552) in hundred_initial_rec_nn:
            newrow = cust_id, 6552, 1

        #diversity check Out of Sight 13486
        elif knn.trainset.to_inner_iid(13486) in hundred_initial_rec_nn:
            newrow = cust_id, 13486, 1

        #diversity check Bad Boys 2186
        elif knn.trainset.to_inner_iid(2186) in hundred_initial_rec_nn:
            newrow = cust_id, 2186, 1

        #else return MIB 12918
        else:
            new_row = cust_id, 12918, 1

        #insert new row at index 3
        initial_recs_df.loc[3] = new_row
        return initial_recs_df[:10]

In [167]:
diversity(svdpp_example, list1)

/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


,cust_id,mid,m_minreq
267998,2407458,12834,0.0
502332,2407458,7230,0.0
864390,2407458,10418,0.0
997165,2407458,16587,0.0
1151834,2407458,14621,0.0
1558123,2407458,12293,0.0
1713263,2407458,2102,0.0
2183369,2407458,5103,0.0
2553679,2407458,14691,0.0
2824970,2407458,10080,0.0


In [ ]:
v2v_df = pd.read_csv('data/v2vrecs.csv')


## Personalized Video Ranker

This is 

In [56]:
#DO NOT TOUCH


#DO NOT TOUCH

def pvr(cust_id):
    all_recs = []

    def rec_pvr(cust_id):

        # list of all content user has not rated
        all_content = data['mid'].unique()
        user_content = data[data['cust_id'] == cust_id]['mid'].unique()
        new_content = list(set(all_content) - set(user_content))

        # predict the ratings for new content
        preds = [svdpp.predict(cust_id, mid) for mid in new_content]

        # sort preds by estimated rating
        preds.sort(key=lambda pred: pred.est, reverse=True)

        # top 10 recommendations
        top_10 = [prediction.iid for prediction in preds[:10]]
        
        #return list of cust,mid pairs
        return [(cust_id, movie_id) for movie_id in top_10]
    
    #apply rec_pvr fxn and + to list
    user_recs = rec_pvr(cust_id)
    all_recs.extend(user_recs)

    #new df of recs for analysis including minority requirement
    recs = pd.DataFrame(all_recs, columns=["cust_id", "mid"])
    recs = recs.merge(minorityrec)
    
    return recs

In [57]:
pvr1 = pvr(2407458)
pvr1

,cust_id,mid,m_minreq
0,2407458,17157,0.0
1,2407458,17085,0.0
2,2407458,6007,0.0
3,2407458,5760,0.0
4,2407458,14961,0.0
5,2407458,12242,0.0
6,2407458,7742,0.0
7,2407458,2129,0.0
8,2407458,5103,0.0
9,2407458,4877,0.0


In [106]:
mids = list(pvr1['mid'].unique())
mids

[17157, 17085, 6007, 5760, 14961, 12242, 7742, 2129, 5103, 4877]

In [107]:
pvr9 = pvr1[:10]
pvr9

,cust_id,mid,m_minreq
0,2407458,17157,0.0
1,2407458,17085,0.0
2,2407458,6007,0.0
3,2407458,5760,0.0
4,2407458,14961,0.0
5,2407458,12242,0.0
6,2407458,7742,0.0
7,2407458,2129,0.0
8,2407458,5103,0.0
9,2407458,4877,0.0


In [77]:
#top 10 does it contain a 1? 

def pvr(cust_id):
    
    diverse_recs = []
    
    #get initial recommendations
    def cust_id_initial_recs(cust_id):
    
        first_recs = []

        def rec_pvr(cust_id):

            # list of all content user has not rated
            all_content = data['mid'].unique()
            user_content = data[data['cust_id'] == cust_id]['mid'].unique()
            new_content = list(set(all_content) - set(user_content))

            # predict the ratings for new content
            preds = [svdpp.predict(cust_id, mid) for mid in new_content]

            # sort preds by estimated rating
            preds.sort(key=lambda pred: pred.est, reverse=True)

            # top 10 recommendations
            top_10 = [prediction.iid for prediction in preds[:10]]

            #return list of cust,mid pairs
            return [(cust_id, movie_id) for movie_id in top_10]

        #apply rec_pvr fxn and + to list
        user_recs = rec_pvr(cust_id)
        first_recs.extend(user_recs)

        #new df of recs for analysis including minority requirement
        recs = pd.DataFrame(all_recs, columns=["cust_id", "mid"])
        recs = recs.merge(minorityrec)
        
        return recs

    def diversity_check(initial_recs_df):
        '''
        if recs df meets minority req, will return df as is
        else, will check if nns are in common with diverse dict
            if there's one in common, it will return that movie in rec list
        if still none,
            will return Men In Black in rec list
        '''
        
        #list of mids in rec
        mid_list = list(initial_recs_df['mid'].unique())
        
        #nearest neighbors for initial recs_df
        def nn_rec_mids(mid_list):
            #instantiate list of 10nn for each rec
            rec_nn_list = []
            
            def nn(mid):
                #inner id
                mid_iid = knn.trainset.to_inner_iid(mid)

                #top 10 neighbors
                neighbors = knn.get_neighbors(mid_iid, k=10)
                
        #apply nn to get list of 100nns
        ten_initial_nn = nn_rec_mids(mid_list)
        rec_nn_list.extend(user_recs)
        
        
    if 1 in data['mid'].unique():
            return recs
        
    else:
        diversity_check(recs)
    
    

In [79]:
pvrx, pvrx1 = rec_pvr_x(2407458)

ValueError: too many values to unpack (expected 2)

In [61]:
test_item = pvr1['mid'][0]
test_item

17157

In [72]:
will1 = 17324
will1 = knn.trainset.to_inner_iid(will1)
neighbor1 = knn.get_neighbors(test_item1, k=20)
neighbors1 = [knn.trainset.to_raw_iid(inner_id) for inner_id in neighbor1]
neighbors1

[7533,
 17251,
 6574,
 14983,
 12303,
 10278,
 13393,
 10773,
 10345,
 15662,
 9746,
 1861,
 9517,
 3640,
 8204,
 2252,
 17134,
 3267,
 9204,
 12262]

In [65]:
#inner id
test_item1 = knn.trainset.to_inner_iid(test_item)
test_item1

528

In [69]:
neighbor = knn.get_neighbors(test_item1, k=20)
neighbor

[5, 6, 11, 12, 14, 22, 27, 37, 43, 44, 49, 50, 53, 54, 57, 66, 71, 72, 74, 76]

In [71]:
neighbors = [knn.trainset.to_raw_iid(inner_id) for inner_id in neighbor]
neighbors

[7533,
 17251,
 6574,
 14983,
 12303,
 10278,
 13393,
 10773,
 10345,
 15662,
 9746,
 1861,
 9517,
 3640,
 8204,
 2252,
 17134,
 3267,
 9204,
 12262]

In [ ]:
def pvr(cust_id):
    all_recs = []

    def rec_pvr(cust_id):

        # list of all content user has not rated
        all_content = data['mid'].unique()
        user_content = data[data['cust_id'] == cust_id]['mid'].unique()
        new_content = list(set(all_content) - set(user_content))

        # predict the ratings for new content
        preds = [svdpp.predict(cust_id, mid) for mid in new_content]

        # sort preds by estimated rating
        preds.sort(key=lambda pred: pred.est, reverse=True)

        # top 10 recommendations
        top_10 = [prediction.iid for prediction in preds[:10]]
        
        #return list of cust,mid pairs
        return [(cust_id, movie_id) for movie_id in top_10]
    
    #apply rec_pvr fxn and + to list
    user_recs = rec_pvr(cust_id)
    all_recs.extend(user_recs)

    #new df of recs for analysis including minority requirement
    recs = pd.DataFrame(all_recs, columns=["cust_id", "mid"])
    recs = recs.merge(minorityrec)
    
    
    
    return recs

In [ ]:
user = pvr(2407458)
user

In [ ]:
def get_recs_bsl(user):
    all_recommendations = []
    
    def rec_content(cust_id):
        # Get a list of all the movies the user has not rated
        all_content = data['mid'].unique()
        user_content = data[data['cust_id'] == cust_id]['mid'].unique()
        new_content = list(set(all_content) - set(user_content))

        # Predict the ratings for the new movies
        predictions = [bias_baseline.predict(cust_id, mid) for mid in new_content]

        # Sort the predictions by estimated rating
        predictions.sort(key=lambda x: x.est, reverse=True)

        # Get the top 10 recommendations
        top_recommendations = [prediction.iid for prediction in predictions[:10]]

        return [(cust_id, movie_id) for movie_id in top_recommendations]

    #for all users apply recs
    for cust_id in user_list:
        user_recs = rec_content(cust_id)
        all_recommendations.extend(user_recs)
    
    #new df of recs for analysis
    recommendations_df = pd.DataFrame(all_recommendations, columns=["cust_id", "mid"])
    return recommendations_df

In [ ]:
def v2v_preds(user_list):
    
    '''
    this is an unpersonalized item based recommendation
    returns a movie that is similar to one that has been watched 
    but doesn't take into account any other user behavior
    '''
    
    v2v = []
    
    def v2v_hybrid(cust_id):
        
        # last movie they rated
        rated = data[data['cust_id'] == cust_id]['mid'].to_list()
        mid = rated[-1]
        
        # neighbors from both models
        neighbors = knn_bsl.get_neighbors(all_sr.to_inner_iid(mid), k=10)
        neighborsd = knn_dec.get_neighbors(all_sr.to_inner_iid(mid), k=10)
        
        #combine lists
        nearest_neighbors = neighbors + [i for i in neighborsd if i not in neighbors]

        return [(cust_id, mid, neighbor) for neighbor in nearest_neighbors]
    
    #for all users get the nearest neighbors to the last movie they rated
    for cust_id in user_list:
        user_recs = v2v_hybrid(cust_id)
        v2v.extend(user_recs)
    
    #new df of recs for analysis
    neighbors_df = pd.DataFrame(v2v, columns=["cust_id", "mid", 'recs'])
    return neighbors_df